### **This is for Deep Seek R1**

In [ ]:
# Install Required Libraries
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.9 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create Aspire_LLM directory inside Google Drive
import os

# Define path
aspire_path = "/content/drive/MyDrive/Aspire_LLM"

# Create directory if it doesn't exist
os.makedirs(aspire_path, exist_ok=True)

# Change current working directory to Aspire_LLM
os.chdir(aspire_path)

# Confirm current directory
!pwd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Aspire_LLM


In [ ]:
# Import Libraries
import json
import os
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import torch

In [ ]:
import json

# Input path
input_path = "/content/drive/MyDrive/Aspire_LLM/qna_data.json"

# Read data
with open(input_path, "r") as f:
    raw_data = json.load(f)

# Convert to prompt-completion format
formatted_data = [
    {"prompt": item["question"], "completion": item["answer"]}
    for item in raw_data
]

# Save to JSONL for training
jsonl_path = "/content/drive/MyDrive/Aspire_LLM/formatted_data.jsonl"
with open(jsonl_path, "w") as f:
    for item in formatted_data:
        f.write(json.dumps(item) + "\n")


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset
import json

# Load tokenizer
model_name = "deepseek-ai/deepseek-coder-1.3b-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Read the formatted JSONL file into list
with open("formatted_data.jsonl", "r") as f:
    data = [json.loads(line) for line in f.readlines()]

# Convert to Dataset and split
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.1)


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

print("⏳ Loading DeepSeek 1.3B model with 8-bit quantization...")

quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

if hasattr(model, "peft_config"):
    print("⚠️ PEFT adapter already loaded. Unloading before applying a new one.")
    model = model.unload()

print("⚙️ Applying LoRA adapter...")

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

print("✅ Model loaded and LoRA applied successfully!")
model.print_trainable_parameters()



# # Load tokenizer and model
# model_name = "deepseek-ai/deepseek-coder-1.3b-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     load_in_8bit=True,
#     device_map="auto",
#     trust_remote_code=True
# )


⏳ Loading DeepSeek 1.3B model with 8-bit quantization...
⚙️ Applying LoRA adapter...
✅ Model loaded and LoRA applied successfully!
trainable params: 1,572,864 || all params: 1,348,044,800 || trainable%: 0.1167


In [ ]:
# Prepare LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
def tokenize(example):
    return tokenizer(
        example["prompt"],
        text_target=example["completion"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/358 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import TrainingArguments, Trainer

output_dir = "/content/drive/MyDrive/Aspire_LLM/deepseek-qna-model"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    learning_rate=2e-5,
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    fp16=True,
    save_total_limit=1,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()


<ipython-input-12-29535e886a09>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arhamjaved867 (arhamjaved867-university-of-sialkot) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,7.413200
20,5.524700
30,3.409000
40,1.545500
50,0.714200
60,0.518000
70,0.616700
80,0.562000
90,0.592600
100,0.521100


TrainOutput(global_step=360, training_loss=0.9772020591629876, metrics={'train_runtime': 673.6973, 'train_samples_per_second': 2.126, 'train_steps_per_second': 0.534, 'total_flos': 5639583194677248.0, 'train_loss': 0.9772020591629876, 'epoch': 4.0})

In [ ]:
trainer.save_model(output_dir)

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Aspire_LLM/deepseek-qna-model-pretrained")
tokenizer.save_pretrained("/content/drive/MyDrive/Aspire_LLM/deepseek-qna-model-tokenizer")
print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `after-models` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `after-model

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

model_path = "/content/drive/MyDrive/Aspire_LLM/deepseek-qna-model-pretrained"

# Load tokenizer from the base model repo
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-vl-1.3b-base", trust_remote_code=True)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-vl-1.3b-base",
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

# Load LoRA fine-tuned weights from your saved folder
model = PeftModel.from_pretrained(base_model, model_path)
model.eval()

# Prompt builder
def build_prompt(question: str) -> str:
    return f"""<|system|>\nYou are a helpful assistant that answers customer questions accurately.\n<|user|>\n{question}\n<|assistant|>"""

# Generate answer function
def get_answer(question: str, max_new_tokens=200):
    prompt = build_prompt(question)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("<|assistant|>")[-1].strip()

# Test example
question = "Do you have country-specific scholarships?"
answer = get_answer(question)

print(f"Question: {question}")
print(f"Answer: {answer}")


ValueError: The checkpoint you are trying to load has model type `multi_modality` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

# This code id for Mistral 7B

In [3]:
!pip install --upgrade gcsfs

  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.2
    Uninstalling gcsfs-2025.3.2:
      Successfully uninstalled gcsfs-2025.3.2


In [5]:
!pip install --upgrade fsspec

In [7]:
# Step 1: Install Required Libraries
!pip install -q bitsandbytes accelerate datasets peft transformers trl

In [8]:
# Step 2: Mount Google Drive (if storing model/data there)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Step 3: Load Libraries
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

In [10]:
# Step 4: Load and Convert Your Data
with open("/content/drive/MyDrive/formatted_data.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)

In [12]:
# Step 5: Load Model and Tokenizer (Mistral-7B)
model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
# Step 6: Apply LoRA
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [14]:
# Step 7: Training Arguments
training_args = TrainingArguments(
    output_dir="/content/mistral-qabot",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    push_to_hub=False,
    report_to="none"
)


In [19]:
from trl import SFTTrainer

# Combine prompt and completion into a single text field
def formatting_func(example):
    return f"{example['prompt']}\n{example['completion']}"

# Map into a new dataset with proper text field
formatted_dataset = dataset.map(lambda x: {"text": formatting_func(x)})

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=training_args,
)


Map:   0%|          | 0/398 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/398 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/398 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/398 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/398 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [20]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss
10,1.964000
20,1.554400
30,1.436000
40,1.332100
50,1.428700
60,1.059700
70,1.017500
80,0.898600
90,0.993600
100,1.045200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=147, training_loss=1.082317777231437, metrics={'train_runtime': 767.3376, 'train_samples_per_second': 1.556, 'train_steps_per_second': 0.192, 'total_flos': 2051156532805632.0, 'train_loss': 1.082317777231437})

In [21]:
# Step 9: Save Final Model to Drive
model.save_pretrained("/content/drive/MyDrive/mistral-lora-qabot")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-lora-qabot")

('/content/drive/MyDrive/mistral-lora-qabot/tokenizer_config.json',
 '/content/drive/MyDrive/mistral-lora-qabot/special_tokens_map.json',
 '/content/drive/MyDrive/mistral-lora-qabot/tokenizer.model',
 '/content/drive/MyDrive/mistral-lora-qabot/added_tokens.json',
 '/content/drive/MyDrive/mistral-lora-qabot/tokenizer.json')

# Now Use the Trained Model

In [22]:
!pip install -q transformers peft accelerate bitsandbytes

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# Quantization config for 8-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True
)

# Load base model with quantization and safe offloading
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto"
)

# Load tokenizer and your fine-tuned LoRA adapter
model_path = "/content/drive/MyDrive/mistral-lora-qabot"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(base_model, model_path)

model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p

In [25]:
def ask_question(question, max_new_tokens=200):
    input_text = question.strip()
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [26]:
question = "What documents are required for a UK study visa?"
answer = ask_question(question)
print("Answer:", answer)


OutOfMemoryError: CUDA out of memory. Tried to allocate 92.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 19147 has 14.73 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 56.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Deploye Trained Model in Hugging Face Repo

In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login
notebook_login()


In [29]:
from huggingface_hub import create_repo, upload_folder

# Create a repo (once)
repo_name = "aspire-mistral-lora-qabot"
# create_repo(repo_name, private=True)  # or private=False to make it public

# Upload the folder
upload_folder(
    folder_path="/content/drive/MyDrive/mistral-lora-qabot",
    repo_id=f"Arham-1/{repo_name}",
    path_in_repo=".",  # root
    commit_message="Initial LoRA fine-tuned model upload"
)


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Arham-1/aspire-mistral-lora-qabot/commit/e589df6ecc37603f7e25c53418038b56e0d068dd', commit_message='Initial LoRA fine-tuned model upload', commit_description='', oid='e589df6ecc37603f7e25c53418038b56e0d068dd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Arham-1/aspire-mistral-lora-qabot', endpoint='https://huggingface.co', repo_type='model', repo_id='Arham-1/aspire-mistral-lora-qabot'), pr_revision=None, pr_num=None)

# Code to Make interface to run this model

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes gradio